### Učitavanje potrebnih biblioteka

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from sklearn.metrics import roc_curve
from itertools import combinations
from torch.optim.lr_scheduler import ReduceLROnPlateau
from PIL import Image
import timm
import os
import shutil
import random

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from datetime import datetime
from tqdm.notebook import tqdm


### Učitavanje seta podataka

In [11]:
class FaceDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes

In [ ]:
main_data_folder = '../data/people/'
train_folder = '../data/train/'
valid_folder = '../data/valid/'
test_folder = '../data/test/'

# Remove existing train, valid, and test folders (if they exist)
for folder in [train_folder, valid_folder, test_folder]:
    if os.path.exists(folder):
        shutil.rmtree(folder)  # Delete the folder and its contents

# Create destination folders
for folder in [train_folder, valid_folder, test_folder]:
    os.makedirs(folder, exist_ok=True)

for class_name in os.listdir(main_data_folder):
    class_path = os.path.join(main_data_folder, class_name)
    if not os.path.isdir(class_path):
        continue  # Skip non-folder items

    # List all images in the class folder
    images = os.listdir(class_path)

    # Create class subfolders in train, valid, and test directories
    os.makedirs(os.path.join(train_folder, class_name), exist_ok=True)
    os.makedirs(os.path.join(valid_folder, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_folder, class_name), exist_ok=True)

    # Ensure at least 1 image in test and 1 in validation
    test_images = images[:15]  # First image for testing
    valid_images = images[15:30]  # Second image for validation
    train_images = images[30:]  # Remaining images for training

    # Move images to respective folders
    for img in train_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_folder, class_name, img))
    for img in valid_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(valid_folder, class_name, img))
    for img in test_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(test_folder, class_name, img))

    print(f"Class {class_name}: {len(train_images)} train, {len(valid_images)} valid, {len(test_images)} test images.")

In [5]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [6]:
train_folder = '../data/train/'
valid_folder = '../data/valid/'
test_folder = '../data/test/'

train_dataset = FaceDataset(train_folder, transform=transform)
val_dataset = FaceDataset(valid_folder, transform=transform)
test_dataset = FaceDataset(test_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

### Treniranje modela

In [7]:
import torch
import torch.nn as nn

class FaceClassifier(nn.Module):
    def __init__(self, num_classes):
        super(FaceClassifier, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Reduces spatial dimensions by half
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Reduces spatial dimensions by half again
        )
        self.flatten = nn.Flatten()
        self.fc1 = None  # Placeholder for the dynamically defined layer
        self.classifier = None  # Placeholder for the classifier layer
        self.num_classes = num_classes

    def forward(self, x):
        # Pass through feature extractor
        x = self.features(x)

        # Dynamically calculate flattened size for fully connected layer
        if self.fc1 is None:
            flattened_size = x.shape[1] * x.shape[2] * x.shape[3]
            self.fc1 = nn.Linear(flattened_size, 512).to(x.device)
            self.classifier = nn.Linear(512, self.num_classes).to(x.device)

        # Flatten and pass through fully connected layers
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.classifier(x)
        return x


In [8]:
num_classes = 17
model = FaceClassifier(num_classes)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

### Training loop

In [ ]:

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer)

train_losses, val_losses = [], []
best_val_loss = float('inf')

# Ensure the models directory exists
os.makedirs('../models/', exist_ok=True)

# Dynamic epoch control
while True:
    try:
        # Ask the user how many epochs to run
        num_epochs = int(input("How many epochs would you like to train for? (Enter 0 to stop): "))
        if num_epochs <= 0:
            print("Training stopped by user.")
            break

        for epoch in range(1, num_epochs + 1):
            # Training phase
            model.train()
            running_loss = 0.0
            for images, labels in tqdm(train_loader, desc=f'Epoch {epoch} - Training loop'):
                # Move inputs and labels to the device
                images, labels = images.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * labels.size(0)
            
            train_loss = running_loss / len(train_loader.dataset)
            train_losses.append(train_loss)

            # Validation phase
            model.eval()
            running_loss = 0.0
            with torch.no_grad():
                for images, labels in tqdm(val_loader, desc=f'Epoch {epoch} - Validation loop'):
                    # Move inputs and labels to the device
                    images, labels = images.to(device), labels.to(device)

                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    running_loss += loss.item() * labels.size(0)
            
            val_loss = running_loss / len(val_loader.dataset)
            val_losses.append(val_loss)

            # Print progress
            print(f"Epoch {epoch}/{num_epochs} - Train loss: {train_loss:.4f}, Validation loss: {val_loss:.4f}")

            # Save the best model
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                model_path = f"../models/model_{timestamp}.pth"
                torch.save(model.state_dict(), model_path)
                print(f"New best model saved at {model_path}")

            # Update the scheduler
            scheduler.step(val_loss)

        print(f"Completed {num_epochs} epochs.")
    
    except ValueError:
        print("Invalid input. Please enter a valid number.")

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.title("Loss over epochs")
plt.show()

In [ ]:
import random
import matplotlib.pyplot as plt
import torch
import torchvision
import numpy as np

# Function to display an image
def imshow(img, title=None):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis("off")
    if title:
        plt.title(title, fontsize=12)
    plt.show()

# Get 5 random images from the test_loader
def get_random_images(data_loader, num_images=5):
    data_iter = iter(data_loader)
    images, labels = next(data_iter)  # Get a batch
    indices = random.sample(range(len(images)), num_images)  # Select `num_images` random indices
    return images[indices], labels[indices]

# Load class names from the training folder
train_folder = "../data/people/"
classes = sorted([d for d in os.listdir(train_folder) if os.path.isdir(os.path.join(train_folder, d))])
print(f"Classes: {classes}")

# Get random images and their labels
random_images, true_labels = get_random_images(test_loader, num_images=5)

# Display images and ground truth
grid = torchvision.utils.make_grid(random_images, nrow=5, padding=2, normalize=True)
imshow(grid, title="Ground Truth: " + ', '.join(classes[label] for label in true_labels))

# Model predictions
random_images = random_images.to(device)
true_labels = true_labels.to(device)

model.eval()
with torch.no_grad():
    outputs = model(random_images)
    _, predicted = torch.max(outputs, 1)

# Display predicted labels
print('Predicted: ', ', '.join(f'{classes[predicted[j]]}' for j in range(len(predicted))))


In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

def preprocess_image(image_path, transform):
    image = Image.open(image_path).convert("RGB")
    return image, transform(image).unsqueeze(0)

def predict(model, image_tensor, device):
    model.eval()
    with torch.no_grad():
        image_tensor = image_tensor.to(device)
        outputs = model(image_tensor)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
    return probabilities.cpu().numpy().flatten()

# Visualization
def visualize_predictions(original_image, probabilities, class_names):
    fig, axarr = plt.subplots(1, 2, figsize=(14, 7))
    
    # Display image
    axarr[0].imshow(original_image)
    axarr[0].axis("off")
    
    # Display predictions
    axarr[1].barh(class_names, probabilities)
    axarr[1].set_xlabel("Probability")
    axarr[1].set_title("Class Predictions")
    axarr[1].set_xlim(0, 1)

    plt.tight_layout()
    plt.show()

# Example usage
image_paths = [os.path.join("../data/people/Robert Downey Jr", f) for f in os.listdir("../data/people/Robert Downey Jr")]

for i in range(5):
    original_image, image_tensor = preprocess_image(image_paths[i], transform)

    # Make predictions
    probabilities = predict(model, image_tensor, device)

    # Visualize the predictions
    visualize_predictions(original_image, probabilities, classes)


In [ ]:
import os
import random
from tqdm import tqdm

def generate_pairs(test_folder, valid_folder, max_pairs=5000):
    positive_pairs = []
    negative_pairs = []
    
    # Combine test and validation folders
    all_folders = [test_folder, valid_folder]
    
    # Get the classes from the test and validation folders
    class_folders = {}
    for folder in all_folders:
        class_folders[folder] = os.listdir(folder)
    
    # Create a dictionary to combine test and validation images for each class
    combined_classes = {}
    for folder in all_folders:
        for class_name in class_folders[folder]:
            class_path = os.path.join(folder, class_name)
            images = os.listdir(class_path)
            
            if class_name not in combined_classes:
                combined_classes[class_name] = []
            combined_classes[class_name].extend([os.path.join(class_path, img) for img in images])
    
    # Now we have the combined images for each class, let's generate pairs
    for _ in tqdm(range(max_pairs), desc="Generating pairs"):
        # Randomly choose a class that has images from both test and validation
        class_name = random.choice(list(combined_classes.keys()))
        images = combined_classes[class_name]

        # Create positive pair (same class, one image from test, one from validation)
        if len(images) > 1:
            img1, img2 = random.sample(images, 2)  # Randomly select 2 images for a positive pair
            positive_pairs.append((img1, img2))
        
        # Create negative pair (different classes)
        other_class_name = random.choice([name for name in combined_classes if name != class_name])
        other_class_images = combined_classes[other_class_name]
        
        if len(other_class_images) > 0:
            img1 = random.choice(images)  # Randomly select one image from the current class
            img2 = random.choice(other_class_images)  # Randomly select one image from the other class
            negative_pairs.append((img1, img2))
    
    print(f"Generated {len(positive_pairs)} positive pairs and {len(negative_pairs)} negative pairs.")
    return positive_pairs, negative_pairs

positive_pairs, negative_pairs = generate_pairs(test_folder, valid_folder, max_pairs=1000)

In [22]:
def test_model(model, positive_pairs, negative_pairs, transform, threshold=0.5):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.eval()

    all_labels = []
    all_scores = []

    def load_image(image_path):
        from PIL import Image
        image = Image.open(image_path).convert('RGB')
        return transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        for pair, label in [(positive_pairs, 1), (negative_pairs, 0)]:
            for img1_path, img2_path in tqdm(pair, desc=f"Testing {'Positive' if label == 1 else 'Negative'} Pairs"):
                img1 = load_image(img1_path)
                img2 = load_image(img2_path)

                # Compute embeddings
                embedding1 = model(img1)
                embedding2 = model(img2)

                # Calculate distance
                distance = F.pairwise_distance(embedding1, embedding2).item()
                score = 1 - distance  # Higher score for closer embeddings

                all_scores.append(score)
                all_labels.append(label)

    # Calculate FAR, FRR, and EER
    fpr, tpr, thresholds = roc_curve(all_labels, all_scores)
    fnr = 1 - tpr

    # Calculate EER
    eer_threshold = thresholds[(np.abs(fpr - fnr)).argmin()]
    eer = fpr[(np.abs(fpr - fnr)).argmin()]
    return fpr, fnr, eer, eer_threshold

In [ ]:
# Test the model
fpr, fnr, eer, eer_threshold = test_model(model, positive_pairs, negative_pairs, transform)

# Plot ROC Curve
plt.figure()
plt.plot(fpr, 1 - fnr, label="ROC Curve")
plt.scatter([eer], [1 - eer], color='red', label=f"EER: {eer:.2f}")
plt.xlabel("False Positive Rate (FAR)")
plt.ylabel("True Positive Rate (1 - FRR)")
plt.title("ROC Curve")
plt.legend()
plt.grid()
plt.show()

print(f"Equal Error Rate (EER): {eer}")
print(f"Threshold at EER: {eer_threshold}")